## Setup

In [46]:
import os, sys, re, json, time, unittest, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from pandas_ml import ConfusionMatrix
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [22]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexanderherring/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Note:** The data below is only on the 10k datasets for now. This will be updated to leverage the full datasets. 

# Loading Data

In [2]:
# All types of reviews - 10K dataset
# reviews_df = pd.read_json("../dataset/review_10k.json", lines=True)

# Just restaurant reviews - 10K dataset
reviews_df = pd.read_json("../dataset/restaurant_reviews_10k.json", lines=True)

# All types of reviews
# reviews_df = pd.read_json("../../../final_project/full_dataset/review.json", lines=True)



In [3]:
reviews_df.head()

,business_id,cool,date,funny,stars,text,useful,user_id
0,--6MefnULPED_I942VcFNA,0,2017-08-17,0,4,This is one of my top 3 places to get BBQ pork...,2,FEg8v92qx3kK4Hu4TF28Fg
1,--6MefnULPED_I942VcFNA,0,2017-05-31,0,3,This restaurant is famous for their BBQ dishes...,0,HPtjvIrhzAUkKsiVkeT4MA
2,--6MefnULPED_I942VcFNA,0,2016-10-23,0,2,Roasted pork is one of my favorite things... A...,1,MpvqV7lQcl15rflTBEUhXA
3,--6MefnULPED_I942VcFNA,0,2017-07-30,0,2,I walked by the restaurant more than 5 years a...,1,x-Gbs8sVid3yhJIoHD6Gfw
4,--6MefnULPED_I942VcFNA,0,2017-02-07,1,2,I came here to order a roast duck over rice to...,0,7Dykd1HolQx8mKPYhYDYSg


In [4]:
reviews_df.shape

(10000, 8)

In [5]:
final_reviews_train_df = pd.read_json("../../full_dataset/restaurant_reviews_final_train.json", lines=True)


In [6]:
final_reviews_train_df.head()

,business_id,cool,date,funny,stars,text,useful,user_id
0,--9e1ONYQuAa-CB_Rrw7Tw,0,2008-11-11,0,4,So when you go to a restaurant like this pleas...,0,---1lKK3aKOuomHnwAkAow
1,1JgaRBX0oiRsvEhHF3ZMjw,1,2011-02-16,7,1,Food was 30 mins late and the pizza guy thinks...,5,---1lKK3aKOuomHnwAkAow
2,2BbFeotL85cIaBjSq1SWiA,1,2010-10-17,1,1,When you say your a vegetarian don't recomend ...,2,---1lKK3aKOuomHnwAkAow
3,5aeR9KcboZmhDZlFscnYRA,0,2013-01-16,0,5,So Fresh Mama let us host a Homeschool Board G...,1,---1lKK3aKOuomHnwAkAow
4,5cbsjFtrntUAeUx51FaFTg,0,2010-11-05,0,1,So I was coming here once a month or so maybe ...,1,---1lKK3aKOuomHnwAkAow


In [7]:
final_reviews_train_df.shape

(1500464, 8)

In [8]:
final_reviews_test_df = pd.read_json("../../full_dataset/restaurant_reviews_final_test.json", lines=True)

In [9]:
final_reviews_test_df.head()

,business_id,cool,date,funny,stars,text,useful,user_id
0,7dHYudt6OOIjiaxkSvv3lQ,197,2017-11-08,179,5,Auf unserer Rundreise haben wir häufig die Res...,203,--2vR0DIsmQ6WfcSzKWigw
1,9edPSkfXKsJmkZYIaOmA7Q,194,2017-11-06,176,4,Nachdem wir die Las Vegas North Premium Outlet...,199,--2vR0DIsmQ6WfcSzKWigw
2,El4FC8jcawUVgw_0EIcbaQ,122,2015-10-13,109,4,Das MGM Grand ist mit seinen 6.853 Zimmern ein...,130,--2vR0DIsmQ6WfcSzKWigw
3,GHS1rVjO-RMcRB6WJLpCDQ,108,2015-09-16,94,3,In Las Vegas kann man zwischen zwei verschiede...,112,--2vR0DIsmQ6WfcSzKWigw
4,IhlKa2x5J4vr47hjDY8Jnw,11,2013-06-02,7,5,Bei einer meiner letzten Besuche in Stuttgart ...,15,--2vR0DIsmQ6WfcSzKWigw


In [10]:
final_reviews_test_df.shape

(373155, 8)

# Baseline Model

### Count Vectorizer

In [11]:
n_features = 10000

# text = reviews_df["text"]
train_count_vect = final_reviews_train_df["text"]
test_count_vect = final_reviews_test_df["text"]

print("Fitting Count Vectorizer")
# vectorizer = CountVectorizer(max_df=0.95, min_df=2,
#                                 max_features=n_features,
#                                 stop_words='english')
# word_vector = vectorizer.fit_transform(text)

# No setting of hyper-parameters
vectorizer = CountVectorizer()
word_vector = vectorizer.fit_transform(text)

print(np.shape(word_vector))

Fitting Count Vectorizer
(10000, 24872)


In [12]:
# #Print example text, stars, and embeddings

# print(reviews_df["text"][102])
# print(reviews_df["stars"][102])
# print(word_vector[102])

### GloVe

In [25]:
#https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

embedding_dim = 300
vocabulary_size = 10000
seq_size = 400

# Primary GloVe file
gloveFile = "../../glove/glove.6B.300d.txt"

# Large GloVe file
# gloveFile = "../../glove/glove.42B.300d.txt"

start_time = datetime.datetime.now()
print("Start time: ", start_time)

embeddings_index = {}
f = open(gloveFile)
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



# This function creates a normalized vector for the whole sentence
def sent2vec(s):
#     words = str(s).lower().decode('utf-8')
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())
# create sentence vectors using the above function for training and validation set
x_train_glove = [sent2vec(x) for x in tqdm(final_reviews_train_df["text"])]
x_test_glove = [sent2vec(x) for x in tqdm(final_reviews_test_df["text"])]


x_train_glove = np.array(x_train_glove)
x_test_glove = np.array(x_test_glove)





end_time = datetime.datetime.now()
print("End time: ", end_time)

time_taken = end_time - start_time
print("Time taken: ", time_taken)




0it [00:00, ?it/s]


1324it [00:00, 13009.37it/s]




Start time:  2018-08-07 01:31:15.868949


2628it [00:00, 13036.20it/s]


3972it [00:00, 13183.21it/s]


5374it [00:00, 13395.11it/s]


6792it [00:00, 13546.39it/s]


8174it [00:00, 13599.88it/s]


9510it [00:00, 13559.86it/s]


10849it [00:00, 13539.11it/s]


12161it [00:00, 13493.07it/s]


13460it [00:01, 13440.26it/s]


14752it [00:01, 13269.37it/s]


16008it [00:01, 13209.03it/s]


17265it [00:01, 13160.76it/s]


18521it [00:01, 13104.22it/s]


19834it [00:01, 13105.61it/s]


21104it [00:01, 13066.50it/s]


22368it [00:01, 12971.23it/s]


23634it [00:01, 12955.96it/s]


24886it [00:01, 12931.27it/s]


26131it [00:02, 12889.58it/s]


27367it [00:02, 12852.44it/s]


28638it [00:02, 12843.85it/s]


29887it [00:02, 12827.78it/s]


31139it [00:02, 12816.26it/s]


32385it [00:02, 12795.76it/s]


33674it [00:02, 12799.16it/s]


34961it [00:02, 12801.49it/s]


36226it [00:02, 12772.48it/s]


37472it [00:02, 12751.09it/s]


38763it [00:03, 12756.05it/s]


40035it [00:03, 12753.33it/s]


41293it [00:03, 12711.65it/s]


42587it [00:03

261339it [00:20, 12940.53it/s]


262606it [00:20, 12937.23it/s]


263861it [00:20, 12933.25it/s]


265195it [00:20, 12935.06it/s]


266465it [00:20, 12933.99it/s]


267735it [00:20, 12931.82it/s]


269000it [00:20, 12927.31it/s]


270284it [00:20, 12926.68it/s]


271640it [00:21, 12929.96it/s]


273006it [00:21, 12933.41it/s]


274377it [00:21, 12936.98it/s]


275747it [00:21, 12940.32it/s]


277088it [00:21, 12940.31it/s]


278414it [00:21, 12939.68it/s]


279728it [00:21, 12937.12it/s]


281019it [00:21, 12934.36it/s]


282294it [00:21, 12928.18it/s]


283538it [00:21, 12924.43it/s]


284773it [00:22, 12921.36it/s]


286056it [00:22, 12921.01it/s]


287303it [00:22, 12913.11it/s]


288517it [00:22, 12907.10it/s]


289778it [00:22, 12905.80it/s]


291059it [00:22, 12905.24it/s]


292295it [00:22, 12901.65it/s]


293647it [00:22, 12904.27it/s]


294939it [00:22, 12904.28it/s]


296212it [00:22, 12898.02it/s]


297454it [00:23, 12895.84it/s]


298694it [00:23, 12892.11it/s]


299943it [

Found 400000 word vectors.


...

 86%|████████▌ | 320874/373155 [09:55<01:37, 538.60it/s]


 86%|████████▌ | 320952/373155 [09:55<01:36, 538.60it/s]


 86%|████████▌ | 321023/373155 [09:56<01:36, 538.53it/s]


 86%|████████▌ | 321083/373155 [09:56<01:36, 538.47it/s]


 86%|████████▌ | 321143/373155 [09:56<01:36, 538.48it/s]


 86%|████████▌ | 321226/373155 [09:56<01:36, 538.53it/s]


 86%|████████▌ | 321290/373155 [09:56<01:36, 538.55it/s]


 86%|████████▌ | 321353/373155 [09:56<01:36, 538.55it/s]


 86%|████████▌ | 321418/373155 [09:56<01:36, 538.57it/s]


 86%|████████▌ | 321485/373155 [09:56<01:35, 538.59it/s]


 86%|████████▌ | 321562/373155 [09:56<01:35, 538.63it/s]


 86%|████████▌ | 321646/373155 [09:57<01:35, 538.68it/s]


 86%|████████▌ | 321719/373155 [09:57<01:35, 538.66it/s]


 86%|████████▌ | 321798/373155 [09:57<01:35, 538.70it/s]


 86%|████████▋ | 321867/373155 [09:57<01:35, 538.70it/s]


 86%|████████▋ | 321939/373155 [09:57<01:35, 538.73it/s]


 86%|████████▋ | 322006/373155 [09:57<01:34, 538.71it/s]

 90%|█████████ | 336084/373155 [10:23<01:08, 539.03it/s]


 90%|█████████ | 336139/373155 [10:23<01:08, 539.01it/s]


 90%|█████████ | 336191/373155 [10:23<01:08, 539.00it/s]


 90%|█████████ | 336264/373155 [10:23<01:08, 539.03it/s]


 90%|█████████ | 336327/373155 [10:23<01:08, 539.04it/s]


 90%|█████████ | 336413/373155 [10:24<01:08, 539.09it/s]


 90%|█████████ | 336480/373155 [10:24<01:08, 539.10it/s]


 90%|█████████ | 336545/373155 [10:24<01:07, 539.11it/s]


 90%|█████████ | 336609/373155 [10:24<01:07, 539.12it/s]


 90%|█████████ | 336674/373155 [10:24<01:07, 539.14it/s]


 90%|█████████ | 336737/373155 [10:24<01:07, 539.15it/s]


 90%|█████████ | 336800/373155 [10:24<01:07, 539.16it/s]


 90%|█████████ | 336863/373155 [10:24<01:07, 539.13it/s]


 90%|█████████ | 336938/373155 [10:24<01:07, 539.17it/s]


 90%|█████████ | 337000/373155 [10:25<01:07, 539.17it/s]


 90%|█████████ | 337077/373155 [10:25<01:06, 539.20it/s]


 90%|█████████ | 337142/373155 [10:25<01:06, 539.22it/s]

 94%|█████████▍| 351870/373155 [10:51<00:39, 539.85it/s]


 94%|█████████▍| 351942/373155 [10:51<00:39, 539.86it/s]


 94%|█████████▍| 352029/373155 [10:52<00:39, 539.91it/s]


 94%|█████████▍| 352102/373155 [10:52<00:38, 539.93it/s]


 94%|█████████▍| 352174/373155 [10:52<00:38, 539.92it/s]


 94%|█████████▍| 352251/373155 [10:52<00:38, 539.96it/s]


 94%|█████████▍| 352320/373155 [10:52<00:38, 539.96it/s]


 94%|█████████▍| 352385/373155 [10:52<00:38, 539.95it/s]


 94%|█████████▍| 352450/373155 [10:52<00:38, 539.97it/s]


 94%|█████████▍| 352512/373155 [10:52<00:38, 539.96it/s]


 94%|█████████▍| 352572/373155 [10:52<00:38, 539.97it/s]


 94%|█████████▍| 352631/373155 [10:53<00:38, 539.96it/s]


 95%|█████████▍| 352695/373155 [10:53<00:37, 539.97it/s]


 95%|█████████▍| 352753/373155 [10:53<00:37, 539.95it/s]


 95%|█████████▍| 352819/373155 [10:53<00:37, 539.97it/s]


 95%|█████████▍| 352876/373155 [10:53<00:37, 539.92it/s]


 95%|█████████▍| 352930/373155 [10:53<00:37, 539.92it/s]

 99%|█████████▊| 367653/373155 [11:20<00:10, 539.91it/s]


 99%|█████████▊| 367723/373155 [11:21<00:10, 539.91it/s]


 99%|█████████▊| 367789/373155 [11:21<00:09, 539.92it/s]


 99%|█████████▊| 367854/373155 [11:21<00:09, 539.94it/s]


 99%|█████████▊| 367938/373155 [11:21<00:09, 539.98it/s]


 99%|█████████▊| 368037/373155 [11:21<00:09, 540.05it/s]


 99%|█████████▊| 368116/373155 [11:21<00:09, 540.04it/s]


 99%|█████████▊| 368194/373155 [11:21<00:09, 540.07it/s]


 99%|█████████▊| 368275/373155 [11:21<00:09, 540.11it/s]


 99%|█████████▊| 368353/373155 [11:21<00:08, 540.15it/s]


 99%|█████████▊| 368429/373155 [11:22<00:08, 540.14it/s]


 99%|█████████▉| 368498/373155 [11:22<00:08, 540.14it/s]


 99%|█████████▉| 368563/373155 [11:22<00:08, 540.12it/s]


 99%|█████████▉| 368623/373155 [11:22<00:08, 540.12it/s]


 99%|█████████▉| 368682/373155 [11:22<00:08, 540.06it/s]


 99%|█████████▉| 368732/373155 [11:22<00:08, 539.99it/s]


 99%|█████████▉| 368776/373155 [11:23<00:08, 539.93it/s]

End time:  2018-08-07 02:29:06.364519
Time taken:  0:57:50.495570


## NB Training and Scoring

In [26]:
# x_train_user_reviews = reviews_df["text"][0:6000]
# x_dev_user_reviews = reviews_df["text"][6001:8000]
# x_test_user_reviews = reviews_df["text"][8001:10000]


# x_train_user_reviews = word_vector[0:6000]
# # x_dev_user_reviews = word_vector[6001:8000]
# x_train_user_reviews = word_vector[0:8000]
# x_test_user_reviews = word_vector[8001:10000]

# # print("x_train_user_reviews", x_train_user_reviews)
# # print("shape x_train_user_reviews", np.shape(x_train_user_reviews))



# # y_train_user_stars = reviews_df["stars"][0:6000]
# # y_dev_user_stars = reviews_df["stars"][6001:8000]
# y_train_user_stars = reviews_df["stars"][0:8000]
# y_test_user_stars = reviews_df["stars"][8001:10000]

y_train_user_stars = final_reviews_train_df["stars"]
y_test_user_stars = final_reviews_test_df["stars"]


# self._X_train, self._X_test, self._y_train, self._y_test = train_test_split(padded_seq, target, test_size=0.2, random_state=123)

In [31]:
x_test_glove

array([[ 0.03621468, -0.05508441, -0.05832763, ...,  0.01280963,
         0.04445856,  0.0905762 ],
       [ 0.03270725, -0.04317904, -0.03076379, ...,  0.0322433 ,
         0.03609771,  0.10417826],
       [ 0.02880721, -0.04561727, -0.04301976, ...,  0.02271017,
         0.05601734,  0.09052885],
       ..., 
       [ 0.01458325,  0.00800029,  0.01086005, ..., -0.04563025,
         0.00596677,  0.05798623],
       [-0.02545916,  0.06750739, -0.03254042, ..., -0.05087494,
        -0.02532884,  0.02754192],
       [-0.01357146,  0.04748746, -0.02286445, ..., -0.03602487,
        -0.0113896 ,  0.02273626]])

In [47]:
start_time = datetime.datetime.now()
print("Start time: ", start_time)

# nb = MultinomialNB()
# Using Gaussian NB as it works witht the continuous data from the GloVe implementation
nb = GaussianNB()

# nb.fit(x_train_user_reviews, y_train_user_stars)
nb.fit(x_train_glove, y_train_user_stars)

# y_pred = nb.predict(x_test_user_reviews)
y_pred = nb.predict(x_test_glove)



end_time = datetime.datetime.now()
print("End time: ", end_time)

time_taken = end_time - start_time
print("Time taken: ", time_taken)



Start time:  2018-08-07 04:29:48.705440
End time:  2018-08-07 04:30:10.644833
Time taken:  0:00:21.939393


In [52]:
# acc = accuracy_score(y_pred, y_test_user_stars)
acc = accuracy_score(y_pred, y_test_user_stars)
print("Accuracy on test set: {:.02%}".format(acc))

# log_loss_metric = log_loss(y_test_user_stars, y_pred)
# print("Log-loss on test set: {:.02%}".format(log_loss_metric))

f1 = f1_score(y_test_user_stars, y_pred, average='weighted') 
# print(f1)
print("F1 Score on test set: {:.02}".format(f1))

Accuracy on test set: 35.14%
F1 Score on test set: 0.36


In [2]:
target_names = ['1', '2', '3', '4', '5']
print(classification_report(y_test_user_stars, y_pred, target_names=target_names))

NameError: name 'classification_report' is not defined

In [1]:


# print(ConfusionMatrix(self._eval_actual, self._eval_predicted_classes))
# nb.confusion_matrix.plot(normalized=True)

confus_mat = ConfusionMatrix(y_test_user_stars, y_pred)
confus_mat.plot(normalized=True)


NameError: name 'ConfusionMatrix' is not defined

In [3]:
corr_coeff = matthews_corrcoef(y_true=y_test_user_stars, y_pred=y_pred)
print("corr_coeff:", corr_coeff)

NameError: name 'matthews_corrcoef' is not defined

In [61]:
# Print example prediction

print()
print(y_pred[400])
y_test_user_stars[400]


2


1